In [1]:
import os
import sys

os.environ["PYSPARK_SUBMIT_ARGS"]='--packages com.databricks:spark-csv_2.10:1.2.0 pyspark-shell'
spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, spark_home + "/python")

execfile(os.path.join(spark_home, 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.1.1
      /_/

Using Python version 2.7.6 (default, Oct 26 2016 20:30:19)
SparkSession available as 'spark'.


In [2]:
import json 
import pandas as pd
import numpy as np
import pymorphy2
import pickle
from pyspark.sql import functions as F
from pyspark.sql.functions import col, avg, concat, udf, array
from pyspark.ml.feature import Tokenizer
from pyspark.sql.types import IntegerType, StringType, ArrayType, FloatType, StructType, StructField
from pyspark.ml.feature import Word2Vec

from sklearn.metrics.pairwise import cosine_similarity

import re
import string

In [3]:
df = spark.read.json('/labs/project2/item_details_full')

In [4]:
df = df.fillna('')

In [5]:
df_concat = df.withColumn('concat_text', concat(col('attr0'),col('attr1'))).select('concat_text', 'itemid')

In [6]:
def clearText(text):
    text = text.lower().strip()
    text = re.sub(ur"[^а-яА-ЯёЁ\s]","", text)
    return text
clear = udf(clearText, StringType())
df_clear = df_concat.withColumn('clear_text', clear(df_concat.concat_text)).select('clear_text', 'itemid')

In [7]:
tokenizer = Tokenizer(inputCol='clear_text', outputCol="w_sent")
tokenized = tokenizer.transform(df_clear)
df_sent =tokenized.select("w_sent", "itemid")

In [26]:
for row in df_sent.select('w_sent').take(50):
    for w in row[0]:
        print w


прижизненное
издание
спетербург

год
издание
афдевриена

с

рисунками

типографский
переплет
сохранность
хорошая

наиболее
известное
произведение
американской
писательницы
фбернетт
для
детей

маленький
лорд
фаунтлерой



появилось
в

и
имело
огромный
успех
книга
переведена
на
множество
языков
в
русском
переводе
имеется
несколько
вариантов
названия
в
тч
история
маленького
лорда
и
тп

к
маленькому
лорду
фаунтлерою
жизнь
не
была
сурова
но
на
его
долю
выпало
серьезное
испытание

испытание
богатством
властью
маленький
лорд
сумел
сохранить
доброту
и
благородство
в
самых
тяжелых
условиях

история
маленького
лорда
надежная
защита

благодаря
сверхтонкому
плотному
чехлу



разработанному
в
дополнение
к




производства

достаточно
прикрепить
его
к
обратной
стороне



и
он
в
полной
безопасности

прозрачная
крышка
чехла
с
мягкой
отделкой
не
скрывает
внешний
облик


и
позволяет
удобно
держать
его
в
руках
легкая
конструкция
чехла
подогнанного
под
форму
устройства
оберегает
его
от
повседневного
изно

In [8]:
df_item_pairs = pd.read_pickle('df_item_pairs.pkl')

In [9]:
df_item_pairs = spark.createDataFrame(df_item_pairs)

In [29]:
df_item_pairs.printSchema()

root
 |-- source: string (nullable = true)
 |-- target: string (nullable = true)
 |-- y: long (nullable = true)
 |-- source_ctl: string (nullable = true)
 |-- target_ctl: string (nullable = true)
 |-- source_degree_ctl: long (nullable = true)
 |-- target_degree_ctl: long (nullable = true)
 |-- source_degree_item_ctl: long (nullable = true)
 |-- target_degree_item_ctl: long (nullable = true)
 |-- source_avg_nei_degree: double (nullable = true)
 |-- target_avg_nei_degree: double (nullable = true)
 |-- source_avg_nei_degree_item_ctl: double (nullable = true)
 |-- target_avg_nei_degree_item_ctl: double (nullable = true)
 |-- common_neighbors_ctl: long (nullable = true)
 |-- common_neighbors_item_ctl: long (nullable = true)
 |-- short_path_ctl: long (nullable = true)
 |-- short_path_item_ctl: long (nullable = true)
 |-- y_log: double (nullable = true)



In [10]:
df_s = df_item_pairs.join(df_sent, df_item_pairs.source == df_sent.itemid).selectExpr('source', 'target', 'w_sent as source_w_sent')
df_s = df_s.join(df_sent, df_s.target == df_sent.itemid).selectExpr('source', 'target','source_w_sent', 'w_sent as target_w_sent')

In [38]:
df_s.show()

+--------+--------+--------------------+--------------------+
|  source|  target|       source_w_sent|       target_w_sent|
+--------+--------+--------------------+--------------------+
|27894653| 1021617|[в, первом, томе,...|[издание, , года,...|
| 5168449|  103634|[какой, следует, ...|[ицзин, или, книг...|
|18567295| 1114663|[антония, байетт,...|                  []|
| 1116801| 1116805|                  []|                  []|
| 3090139| 1116805|[представляем, ва...|                  []|
|19088137| 1116805|                  []|                  []|
| 4890774| 1116813|                  []|                  []|
|18189428| 1120025|                  []|                  []|
| 7406547| 1204608|[в, , или, , году...|[когда, ты, собир...|
| 6442022|  121007|[книга, лидии, чу...|[летопись, жизни,...|
| 2447002|  121007|[книга, едобина, ...|[летопись, жизни,...|
| 3671566| 1297213|[приключенческий,...|[писательница, од...|
| 3413255| 1297213|[джеймс, уиллард,...|[писательница, од...|
| 556082

In [39]:
df_s.count()

874020

In [11]:
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors

model = KeyedVectors.load_word2vec_format('web_0_300_20.bin', binary=True)
morph = pymorphy2.MorphAnalyzer()

In [12]:
with open ('stop_words', 'rb') as fp:
    stop_words = pickle.load(fp)

In [13]:
from pyspark.ml.linalg import Vectors, VectorUDT
index2word_set = set(model.index2word)

def text_to_vec(words):
    text_vec = np.zeros((300,), dtype="float32")
    n_words = 0
    

    for word in words:
        parse = morph.parse(word)[0]
        word = parse[2]+'_'+str(parse[1]).split(',')[0]
        if word in index2word_set and not word in stop_words:
            n_words = n_words + 1
            text_vec = np.add(text_vec, model[word])
    if n_words != 0:
        text_vec /= n_words
    return text_vec.tolist()

#text_to_vec(tokenized.limit(1).select('w_sent').collect()[0][0], model)

udf_text_to_vec = udf(text_to_vec, ArrayType(FloatType()))
df_w2v = df_s.withColumn('w2v_source', udf_text_to_vec(df_s.source_w_sent)).select('source', 'target', 'w2v_source','target_w_sent' )
df_w2v = df_w2v.withColumn('w2v_target', udf_text_to_vec(df_w2v.target_w_sent)).select('source', 'target', 'w2v_source', 'w2v_target')

In [ ]:
def cosin(arr):
    return cosine_similarity(arr[0],arr[1])
    
udf_sim = udf(lambda arr: cosin(arr), FloatType())
df_w2v_sim = df_w2v.limit(1).withColumn('sim', udf_sim(array('w2v_source','w2v_target'))).select('source', 'target', 'w2v_source', 'w2v_target', 'sim')

In [ ]:
df_sim = df_w2v_sim.select('source', 'target', 'sim').toPandas()

In [ ]:
df_sim.head()

In [ ]:
df_sim.to_pickle('df_sim.pkl')

In [96]:
!hadoop fs -ls

Found 23 items
drwx------   - roman.smirnov roman.smirnov          0 2017-06-02 23:24 .Trash
drwxrwxrwx   - roman.smirnov roman.smirnov          0 2017-03-30 21:14 .hiveJars
drwxr-xr-x   - roman.smirnov roman.smirnov          0 2017-06-02 21:22 .sparkStaging
drwx------   - roman.smirnov roman.smirnov          0 2017-04-26 03:43 .staging
drwxr-xr-x   - roman.smirnov roman.smirnov          0 2017-04-08 11:41 UserProfile
-rw-r--r--   3 roman.smirnov roman.smirnov  689783692 2017-04-08 11:35 Users
drwxr-xr-x   - roman.smirnov roman.smirnov          0 2017-04-26 03:43 agePred
drwxrwxrwx   - roman.smirnov roman.smirnov          0 2017-03-30 20:54 data
-rw-r--r--   3 roman.smirnov roman.smirnov  655090069 2017-04-03 23:01 gender_age_dataset.txt
-rw-r--r--   3 roman.smirnov roman.smirnov  655090044 2017-04-03 23:10 gender_age_dataset_h.txt
drwxr-xr-x   - roman.smirnov roman.smirnov          0 2017-05-19 15:30 lab09.csv
drwxr-xr-x   - roman.smirnov roman.smirnov          0 2017-05-19 15:46 lab0

In [86]:
!hadoop fs -du -s df_w2v.parquet

0  df_w2v.parquet


In [92]:
!hadoop fs -rm df_w2v.parquet.pkl

rm: `df_w2v.parquet.pkl': No such file or directory


In [27]:
!ls /tmp/bol

vwXKCO.txt


In [28]:
!cp /tmp/bol/vwXKCO.txt ~/project02.txt

BankScoring/          lab07s.json          MapReduseLoadToHBase/
ClassRecomend/        lab08.json           part-00000
hadoop-streaming.jar  lab08s.json          project01_gender-age.csv
HiveUserProfile/      lab09.csv            project02.txt
lab03s_domains.txt    lab09s.csv           Raiting/
lab03_users.txt       lab10.csv            RecSysMovieALS/
lab04.csv             lab10.csv_1          RecSysMovies/
lab04s.csv            lab10s.csv           RecSysProj/
lab05.json            Lesson_1.ipynb       RecSysPurchesFilm/
lab06.json            MapReducePredAge/    Spark/
lab06s.json           MapReduceTop350/     stdout
lab07.json            MapReduceWordCount/  TwoTextCompare_Sentiment/
